### Выбор модели для классификации событий изменения уровня топлива

In [1]:
#imports
import pandas as pd
import numpy as np

from sklearn import model_selection, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

In [2]:
%run "constants-and-functions.ipynb"
%run "problems1_2-preparation.ipynb"

In [3]:
df_X

,dtime_start,dtime_end,delta_seconds,tachometer_start,tachometer_end,delta_fuellevel_abs,delta_fuellevel_sign,lps_abs
0,2020-01-09 10:05:26,2020-01-09 10:06:27,61.0,1248,1056,0.2,1.0,0.003279
1,2020-01-09 10:06:27,2020-01-09 10:07:27,60.0,1056,960,0.4,1.0,0.006667
2,2020-01-09 10:07:27,2020-01-09 10:08:27,60.0,960,864,0.1,1.0,0.001667
3,2020-01-09 10:08:27,2020-01-09 10:09:27,60.0,864,864,0.3,-1.0,0.005000
4,2020-01-09 10:09:27,2020-01-09 10:10:27,60.0,864,864,0.4,-1.0,0.006667
...,...,...,...,...,...,...,...,...
18364,2020-06-27 00:47:05,2020-06-27 00:48:05,60.0,832,832,0.4,1.0,0.006667
18365,2020-06-27 00:48:05,2020-06-27 00:49:05,60.0,832,832,0.2,1.0,0.003333
18366,2020-06-27 00:49:05,2020-06-27 01:15:14,1569.0,832,1408,0.2,-1.0,0.000127
18367,2020-06-27 01:15:14,2020-06-27 01:16:14,60.0,1408,928,0.2,1.0,0.003333


In [4]:
df_Y

,refuel
0,False
1,False
2,False
3,False
4,False
...,...
18364,False
18365,False
18366,False
18367,False


In [5]:
X = df_X[['lps_abs', 'delta_fuellevel_abs', 'delta_seconds', 'tachometer_start', 'tachometer_end']].to_numpy()
Y = df_Y.astype(int)['refuel'].to_numpy()

X

array([[3.27868852e-03, 2.00000000e-01, 6.10000000e+01, 1.24800000e+03,
        1.05600000e+03],
       [6.66666667e-03, 4.00000000e-01, 6.00000000e+01, 1.05600000e+03,
        9.60000000e+02],
       [1.66666667e-03, 1.00000000e-01, 6.00000000e+01, 9.60000000e+02,
        8.64000000e+02],
       ...,
       [1.27469726e-04, 2.00000000e-01, 1.56900000e+03, 8.32000000e+02,
        1.40800000e+03],
       [3.33333333e-03, 2.00000000e-01, 6.00000000e+01, 1.40800000e+03,
        9.28000000e+02],
       [3.33333333e-03, 2.00000000e-01, 6.00000000e+01, 9.28000000e+02,
        8.32000000e+02]])

In [6]:
Y

array([0, 0, 0, ..., 0, 0, 0])

Метрики

In [7]:
scoring_metrics = ['accuracy', 'precision', 'recall', 'f1']

Разбиение на 5 блоков для кросс-валидации

In [8]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

SVM

In [9]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', SVC())
])

Параметры SVM

In [10]:
grid = dict(clf__C=np.power(10.0, np.arange(-3, 2)),
            clf__kernel = ['linear', 'poly', 'rbf'])

Обучение и выбор гиперпараметров

In [12]:
gs = model_selection.GridSearchCV(estimator=pipe,
                                  param_grid=grid,
                                  scoring=scoring_metrics,
                                  refit='recall',
                                  n_jobs=4,
                                  verbose=5)
gs.fit(X, Y)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   15.0s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done  75 out of  75 | elapsed: 12.4min finished


GridSearchCV(estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('clf', SVC())]),
             n_jobs=4,
             param_grid={'clf__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01]),
                         'clf__kernel': ['linear', 'poly', 'rbf']},
             refit='recall', scoring=['accuracy', 'precision', 'recall', 'f1'],
             verbose=5)

In [21]:
results = pd.DataFrame(gs.cv_results_)

results = results[[x for x in results.columns if (x =='params' or 'mean' in x)]]

results

,mean_fit_time,mean_score_time,params,mean_test_accuracy,mean_test_precision,mean_test_recall,mean_test_f1
0,3.932199,0.307823,"{'clf__C': 0.001, 'clf__kernel': 'linear'}",0.642608,0.664094,0.530598,0.582744
1,5.140458,0.481142,"{'clf__C': 0.001, 'clf__kernel': 'poly'}",0.580767,0.870302,0.145413,0.229770
2,6.112951,0.895441,"{'clf__C': 0.001, 'clf__kernel': 'rbf'}",0.644622,0.691984,0.476239,0.555558
3,3.796902,0.298212,"{'clf__C': 0.01, 'clf__kernel': 'linear'}",0.642063,0.659286,0.540513,0.587098
4,5.780113,0.429706,"{'clf__C': 0.01, 'clf__kernel': 'poly'}",0.609129,0.786077,0.248319,0.364035
5,5.989063,0.747132,"{'clf__C': 0.01, 'clf__kernel': 'rbf'}",0.644132,0.690586,0.476923,0.555848
6,4.556443,0.288005,"{'clf__C': 0.1, 'clf__kernel': 'linear'}",0.642335,0.659402,0.541766,0.587861
7,6.864086,0.434109,"{'clf__C': 0.1, 'clf__kernel': 'poly'}",0.621759,0.766451,0.298803,0.418807
8,6.497825,0.753137,"{'clf__C': 0.1, 'clf__kernel': 'rbf'}",0.641900,0.677935,0.494929,0.564047
9,6.374340,0.284400,"{'clf__C': 1.0, 'clf__kernel': 'linear'}",0.642390,0.659453,0.541880,0.587931


In [22]:
gs.best_estimator_

Pipeline(steps=[('scale', StandardScaler()),
                ('clf', SVC(C=10.0, kernel='linear'))])

SVM показывает плохие результаты, мб стоит будет попробовать c PCA
